In [1]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import datasets, ensemble

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import xgboost as xgb
from auto_ts import auto_timeseries
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from pandas.tseries.offsets import BDay

Imported auto_timeseries version:0.0.92. Call by using:
model = auto_timeseries(score_type='rmse',
        time_interval='M', non_seasonal_pdq=None, seasonality=False,
        seasonal_period=12, model_type=['best'], verbose=2, dask_xgboost_flag=0)
model.fit(traindata, ts_column,target)
model.predict(testdata, model='best')



In [2]:
stock_data = pd.read_csv('Kaggle_Data/SP500_data.csv',parse_dates=['Date'],index_col='Date')

stock_data.head()

,Close/Last,Open,High,Low,VIX_Open,VIX_High,VIX_Low,VIX_Close,Oil_Price,Oil_Open,Oil_High,Oil_Low,Oil_Volume,Oil_%Change,VFIX_Price,28_Day_VAR,21_Day_VAR
Date,,,,,,,,,,,,,,,,,
2024-09-11,"5,554.13","5,496.42","5,560.41","5,406.96",19.41,21.41,17.55,17.69,66.60,65.73,67.14,64.99,237.20,0.02,513.72,"18,587.55","6,873.55"
2024-09-12,"5,595.76","5,557.48","5,600.71","5,535.50",17.62,18.59,16.89,17.07,68.15,66.68,68.93,66.55,221.59,0.02,517.58,"18,732.38","4,840.11"
2024-09-13,"5,626.02","5,603.34","5,636.27","5,601.65",17.03,17.18,16.23,16.56,67.75,68.35,69.35,67.58,230.30,-0.01,520.46,"18,410.86","4,195.59"
2024-09-16,"5,633.09","5,615.21","5,636.05","5,604.53",17.16,17.69,16.91,17.14,69.02,68.22,69.61,67.70,235.88,0.02,521.21,"15,326.39","3,814.02"
2024-09-17,"5,634.58","5,655.51","5,670.81","5,614.05",17.16,18.08,16.67,17.61,69.96,69.35,70.65,68.51,289.93,0.01,521.36,"13,020.74","3,984.73"


In [3]:
len(stock_data)

subset_data = stock_data
for i in range(1, 5):
    subset_data[f'Close_lag_{i}'] = subset_data['Close/Last'].shift(i)
    subset_data[f'Open_lag_{i}'] = subset_data['Open'].shift(i)
    subset_data[f'High_lag_{i}'] = subset_data['High'].shift(i)
    subset_data[f'Low_lag_{i}'] = subset_data['Low'].shift(i)
    subset_data[f'VIX_Open_lag_{i}'] = subset_data['VIX_Open'].shift(i)
    subset_data[f'VIX_High_lag_{i}'] = subset_data['VIX_High'].shift(i)
    subset_data[f'VIX_Low_lag_{i}'] = subset_data['VIX_Low'].shift(i)
    subset_data[f'VIX_Close_lag_{i}'] = subset_data['VIX_Close'].shift(i)
    subset_data[f'Oil_Price_lag_{i}'] = subset_data['Oil_Price'].shift(i)
    subset_data[f'Oil_Open_lag_{i}'] = subset_data['Oil_Open'].shift(i)
    subset_data[f'Oil_Volume_lag_{i}'] = subset_data['Oil_Volume'].shift(i)
    subset_data[f'Oil_%Change_lag_{i}'] = subset_data['Oil_%Change'].shift(i)
    subset_data[f'VFIX_Price_lag_{i}'] = subset_data['VFIX_Price'].shift(i)
    subset_data[f'28_Day_VAR_lag_{i}'] = subset_data['28_Day_VAR'].shift(i)
    subset_data[f'21_Day_VAR_lag_{i}'] = subset_data['21_Day_VAR'].shift(i)

subset_data = subset_data.dropna()
subset_data.head()

len(subset_data)

138

,Close/Last,Open,High,Low,VIX_Open,VIX_High,VIX_Low,VIX_Close,Oil_Price,Oil_Open,...,VIX_High_lag_4,VIX_Low_lag_4,VIX_Close_lag_4,Oil_Price_lag_4,Oil_Open_lag_4,Oil_Volume_lag_4,Oil_%Change_lag_4,VFIX_Price_lag_4,28_Day_VAR_lag_4,21_Day_VAR_lag_4
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-17,"5,634.58","5,655.51","5,670.81","5,614.05",17.16,18.08,16.67,17.61,69.96,69.35,...,21.41,17.55,17.69,66.60,65.73,237.20,0.02,513.72,"18,587.55","6,873.55"
2024-09-18,"5,618.26","5,641.68","5,689.75","5,615.08",17.58,19.39,17.11,18.23,69.88,69.98,...,18.59,16.89,17.07,68.15,66.68,221.59,0.02,517.58,"18,732.38","4,840.11"
2024-09-19,"5,713.64","5,702.63","5,733.57","5,686.42",17.21,17.27,16.21,16.33,71.16,69.15,...,17.18,16.23,16.56,67.75,68.35,230.30,-0.01,520.46,"18,410.86","4,195.59"
2024-09-20,"5,702.55","5,709.64","5,715.14","5,674.49",16.35,16.68,15.81,16.15,71.00,71.18,...,17.69,16.91,17.14,69.02,68.22,235.88,0.02,521.21,"15,326.39","3,814.02"
2024-09-23,"5,718.57","5,711.90","5,725.36","5,704.22",16.71,16.95,15.75,15.89,69.61,70.43,...,18.08,16.67,17.61,69.96,69.35,289.93,0.01,521.36,"13,020.74","3,984.73"


134

In [4]:
subset_data.isna().sum()
subset_data.columns

Close/Last           0
Open                 0
High                 0
Low                  0
VIX_Open             0
                    ..
Oil_Volume_lag_4     0
Oil_%Change_lag_4    0
VFIX_Price_lag_4     0
28_Day_VAR_lag_4     0
21_Day_VAR_lag_4     0
Length: 77, dtype: int64

Index(['Close/Last', 'Open', 'High', 'Low', 'VIX_Open', 'VIX_High', 'VIX_Low',
       'VIX_Close', 'Oil_Price', 'Oil_Open', 'Oil_High', 'Oil_Low',
       'Oil_Volume', 'Oil_%Change', 'VFIX_Price', '28_Day_VAR', '21_Day_VAR',
       'Close_lag_1', 'Open_lag_1', 'High_lag_1', 'Low_lag_1',
       'VIX_Open_lag_1', 'VIX_High_lag_1', 'VIX_Low_lag_1', 'VIX_Close_lag_1',
       'Oil_Price_lag_1', 'Oil_Open_lag_1', 'Oil_Volume_lag_1',
       'Oil_%Change_lag_1', 'VFIX_Price_lag_1', '28_Day_VAR_lag_1',
       '21_Day_VAR_lag_1', 'Close_lag_2', 'Open_lag_2', 'High_lag_2',
       'Low_lag_2', 'VIX_Open_lag_2', 'VIX_High_lag_2', 'VIX_Low_lag_2',
       'VIX_Close_lag_2', 'Oil_Price_lag_2', 'Oil_Open_lag_2',
       'Oil_Volume_lag_2', 'Oil_%Change_lag_2', 'VFIX_Price_lag_2',
       '28_Day_VAR_lag_2', '21_Day_VAR_lag_2', 'Close_lag_3', 'Open_lag_3',
       'High_lag_3', 'Low_lag_3', 'VIX_Open_lag_3', 'VIX_High_lag_3',
       'VIX_Low_lag_3', 'VIX_Close_lag_3', 'Oil_Price_lag_3', 'Oil_Open_lag_3',
 

In [5]:
X = subset_data[[f'Close_lag_{i}' for i in range(1, 5)] + 
                [f'Open_lag_{i}' for i in range(1, 5)] + 
                #[f'High_lag_{i}' for i in range(1, 5)] + 
                #[f'Low_lag_{i}' for i in range(1, 5)] + 

                #[f'VIX_High_lag_{i}' for i in range(1, 5)] +
                #[f'VIX_Low_lag_{i}' for i in range(1, 5)] + 
                [f'VIX_Close_lag_{i}' for i in range(1, 5)] + 
                [f'VIX_Open_lag_{i}' for i in range(1, 5)] + 

                [f'Oil_Price_lag_{i}' for i in range(1, 5)] + 
                [f'Oil_Open_lag_{i}' for i in range(1, 5)] + 
                [f'Oil_Volume_lag_{i}' for i in range(1, 5)] + 
                [f'Oil_%Change_lag_{i}' for i in range(1, 5)] + 
                [f'VFIX_Price_lag_{i}' for i in range(1, 5)] + 
                #[f'28_Day_VAR_lag_{i}' for i in range(1, 5)] + 
                #[f'21_Day_VAR_lag_{i}' for i in range(1, 5)] + 

                ['Open']].copy()

y = subset_data['Close/Last']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

X_train
X_test
y_train
y_test

,Close_lag_1,Close_lag_2,Close_lag_3,Close_lag_4,Open_lag_1,Open_lag_2,Open_lag_3,Open_lag_4,VIX_Close_lag_1,VIX_Close_lag_2,...,Oil_Volume_lag_4,Oil_%Change_lag_1,Oil_%Change_lag_2,Oil_%Change_lag_3,Oil_%Change_lag_4,VFIX_Price_lag_1,VFIX_Price_lag_2,VFIX_Price_lag_3,VFIX_Price_lag_4,Open
Date,,,,,,,,,,,,,,,,,,,,,
2024-09-17,"5,633.09","5,626.02","5,595.76","5,554.13","5,615.21","5,603.34","5,557.48","5,496.42",17.14,16.56,...,237.20,0.02,-0.01,0.02,0.02,521.21,520.46,517.58,513.72,"5,655.51"
2024-09-18,"5,634.58","5,633.09","5,626.02","5,595.76","5,655.51","5,615.21","5,603.34","5,557.48",17.61,17.14,...,221.59,0.01,0.02,-0.01,0.02,521.36,521.21,520.46,517.58,"5,641.68"
2024-09-19,"5,618.26","5,634.58","5,633.09","5,626.02","5,641.68","5,655.51","5,615.21","5,603.34",18.23,17.61,...,230.30,-0.00,0.01,0.02,-0.01,519.86,521.36,521.21,520.46,"5,702.63"
2024-09-20,"5,713.64","5,618.26","5,634.58","5,633.09","5,702.63","5,641.68","5,655.51","5,615.21",16.33,18.23,...,235.88,0.02,-0.00,0.01,0.02,528.71,519.86,521.36,521.21,"5,709.64"
2024-09-23,"5,702.55","5,713.64","5,618.26","5,634.58","5,709.64","5,702.63","5,641.68","5,655.51",16.15,16.33,...,289.93,-0.00,0.02,-0.00,0.01,527.69,528.71,519.86,521.36,"5,711.90"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13,"6,051.97","6,068.50","6,066.44","6,025.99","6,025.08","6,049.32","6,046.40","6,083.13",15.89,16.02,...,273.77,-0.03,0.01,0.01,0.01,559.15,560.62,560.42,556.61,"6,060.59"
2025-02-14,"6,115.07","6,051.97","6,068.50","6,066.44","6,060.59","6,025.08","6,049.32","6,046.40",15.10,15.89,...,160.44,-0.00,-0.03,0.01,0.01,564.99,559.15,560.62,560.42,"6,115.52"
2025-02-18,"6,114.63","6,115.07","6,051.97","6,068.50","6,115.52","6,060.59","6,025.08","6,049.32",14.77,15.10,...,195.49,-0.01,-0.00,-0.03,0.01,565.07,564.99,559.15,560.62,"6,121.60"


,Close_lag_1,Close_lag_2,Close_lag_3,Close_lag_4,Open_lag_1,Open_lag_2,Open_lag_3,Open_lag_4,VIX_Close_lag_1,VIX_Close_lag_2,...,Oil_Volume_lag_4,Oil_%Change_lag_1,Oil_%Change_lag_2,Oil_%Change_lag_3,Oil_%Change_lag_4,VFIX_Price_lag_1,VFIX_Price_lag_2,VFIX_Price_lag_3,VFIX_Price_lag_4,Open
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-21,"6,117.52","6,144.15","6,129.58","6,114.63","6,134.50","6,117.76","6,121.60","6,115.52",15.66,15.27,...,213.47,0.00,0.01,0.02,-0.01,565.50,567.89,566.53,565.07,"6,114.10"
2025-02-24,"6,013.13","6,117.52","6,144.15","6,129.58","6,114.10","6,134.50","6,117.76","6,121.60",18.21,15.66,...,130.94,-0.03,0.00,0.01,0.02,555.87,565.50,567.89,566.53,"6,026.69"
2025-02-25,"5,983.25","6,013.13","6,117.52","6,144.15","6,026.69","6,114.10","6,134.50","6,117.76",18.98,18.21,...,69.01,0.00,-0.03,0.00,0.01,553.12,555.87,565.50,567.89,"5,982.73"
2025-02-26,"5,955.25","5,983.25","6,013.13","6,117.52","5,982.73","6,026.69","6,114.10","6,134.50",19.43,18.98,...,26.71,-0.03,0.00,-0.03,0.00,550.54,553.12,555.87,565.50,"5,970.87"
2025-02-27,"5,956.06","5,955.25","5,983.25","6,013.13","5,970.87","5,982.73","6,026.69","6,114.10",19.10,19.43,...,268.47,-0.00,-0.03,0.00,-0.03,550.62,550.54,553.12,555.87,"5,981.88"
2025-02-28,"5,861.57","5,956.06","5,955.25","5,983.25","5,981.88","5,970.87","5,982.73","6,026.69",21.13,19.10,...,205.47,0.03,-0.00,-0.03,0.00,541.89,550.62,550.54,553.12,"5,856.74"
2025-03-03,"5,954.50","5,861.57","5,956.06","5,955.25","5,856.74","5,981.88","5,970.87","5,982.73",19.63,21.13,...,267.31,-0.01,0.03,-0.00,-0.03,550.55,541.89,550.62,550.54,"5,968.33"
2025-03-04,"5,849.72","5,954.50","5,861.57","5,956.06","5,968.33","5,856.74","5,981.88","5,970.87",22.78,19.63,...,255.60,-0.02,-0.01,0.03,-0.00,540.90,550.55,541.89,550.62,"5,811.98"
2025-03-05,"5,778.15","5,849.72","5,954.50","5,861.57","5,811.98","5,968.33","5,856.74","5,981.88",23.51,22.78,...,265.93,-0.00,-0.02,-0.01,0.03,534.30,540.90,550.55,541.89,"5,781.36"


Date
2024-09-17   5,634.58
2024-09-18   5,618.26
2024-09-19   5,713.64
2024-09-20   5,702.55
2024-09-23   5,718.57
               ...   
2025-02-13   6,115.07
2025-02-14   6,114.63
2025-02-18   6,129.58
2025-02-19   6,144.15
2025-02-20   6,117.52
Name: Close/Last, Length: 107, dtype: float64

Date
2025-02-21   6,013.13
2025-02-24   5,983.25
2025-02-25   5,955.25
2025-02-26   5,956.06
2025-02-27   5,861.57
2025-02-28   5,954.50
2025-03-03   5,849.72
2025-03-04   5,778.15
2025-03-05   5,842.63
2025-03-06   5,738.52
2025-03-07   5,770.20
2025-03-10   5,614.56
2025-03-11   5,572.07
2025-03-12   5,599.30
2025-03-13   5,521.52
2025-03-14   5,638.94
2025-03-17   5,675.12
2025-03-18   5,614.66
2025-03-19   5,675.29
2025-03-20   5,662.89
2025-03-21   5,667.56
2025-03-24   5,767.57
2025-03-25   5,776.65
2025-03-26   5,712.20
2025-03-27   5,693.31
2025-03-28   5,580.94
2025-03-31   5,611.85
Name: Close/Last, dtype: float64

In [7]:
closelast_model = LinearRegression(fit_intercept = True)
#closelast_model = Ridge(alpha = 0.1, fit_intercept = True)
closelast_model.fit(X_train, y_train) 

# The following gives the R-square score
closelast_model.score(X_train, y_train) 

# This is the coefficient Beta_1, ..., Beta_7
closelast_model.coef_

# This is the coefficient Beta_0
closelast_model.intercept_

LinearRegression()

0.9477839190326303

array([ 7.12845135e-01, -1.84782203e+00,  9.38747112e-01, -6.13363764e-01,
       -1.81000160e-01, -6.78094689e-03, -1.25438115e-01,  6.59665994e-03,
       -7.57061668e+00,  1.25700440e+01,  2.41456137e+00,  7.54137526e+00,
       -5.31999730e+00, -4.60477731e+00, -3.62105971e-01, -2.94956307e+00,
       -1.73060730e+01, -1.23074190e+01,  2.26968090e+01, -2.25859673e+01,
       -3.45332427e+00,  1.54693978e+01,  1.51260993e+01,  6.68293156e+00,
       -2.62142981e-03, -1.59625234e-02,  2.89000372e-02, -4.05956091e-02,
        1.47884626e+03,  2.38260739e+03, -1.94236837e+02,  3.65348147e+02,
       -1.09936653e+01,  2.45382310e+01, -1.01894354e+01,  8.97745984e+00,
        9.65719566e-01])

-259.8413501139257

In [8]:
training_residuals = y_train - closelast_model.predict(X_train)

In [9]:
#set to 0 for random forest or 1 for gradient boosting
treemodelchoice = 0
treemodel = None
if treemodelchoice == 0:
    treemodel = RandomForestRegressor(n_estimators=600, random_state=50, min_samples_leaf = 2, max_features = "sqrt")
    treemodel = treemodel.fit(X_train, training_residuals) 
    X_train.columns
    treemodel.feature_importances_
else:
    treemodel = GradientBoostingRegressor(n_estimators=500,random_state=50, min_samples_leaf = 2, max_depth = 8)
    treemodel = treemodel.fit(X_train, training_residuals)
    X_train.columns
    treemodel.feature_importances_


Index(['Close_lag_1', 'Close_lag_2', 'Close_lag_3', 'Close_lag_4',
       'Open_lag_1', 'Open_lag_2', 'Open_lag_3', 'Open_lag_4',
       'VIX_Close_lag_1', 'VIX_Close_lag_2', 'VIX_Close_lag_3',
       'VIX_Close_lag_4', 'VIX_Open_lag_1', 'VIX_Open_lag_2', 'VIX_Open_lag_3',
       'VIX_Open_lag_4', 'Oil_Price_lag_1', 'Oil_Price_lag_2',
       'Oil_Price_lag_3', 'Oil_Price_lag_4', 'Oil_Open_lag_1',
       'Oil_Open_lag_2', 'Oil_Open_lag_3', 'Oil_Open_lag_4',
       'Oil_Volume_lag_1', 'Oil_Volume_lag_2', 'Oil_Volume_lag_3',
       'Oil_Volume_lag_4', 'Oil_%Change_lag_1', 'Oil_%Change_lag_2',
       'Oil_%Change_lag_3', 'Oil_%Change_lag_4', 'VFIX_Price_lag_1',
       'VFIX_Price_lag_2', 'VFIX_Price_lag_3', 'VFIX_Price_lag_4', 'Open'],
      dtype='object')

array([0.01707693, 0.02075434, 0.01886615, 0.01989037, 0.02132841,
       0.02279462, 0.02821088, 0.03308769, 0.0226961 , 0.02299716,
       0.02507446, 0.03100695, 0.01953485, 0.02680096, 0.05043906,
       0.05000094, 0.02366241, 0.02965288, 0.03028892, 0.01948204,
       0.02715001, 0.02371529, 0.02117137, 0.02647216, 0.03456809,
       0.01878381, 0.03271016, 0.04523981, 0.03373523, 0.0267587 ,
       0.03411518, 0.02853142, 0.02001588, 0.02585841, 0.01949154,
       0.02340279, 0.02463405])

In [10]:
pred_residuals = treemodel.predict(X_test)
y_pred = pred_residuals + closelast_model.predict(X_test)

In [11]:
test_output = pd.DataFrame(y_pred, index = X_test.index, columns = ['pred_Close/Last'])
test_output = test_output.merge(y_test, left_index = True, right_index = True)
test_output.tail()
mean_absolute_error = abs(test_output['pred_Close/Last'] - test_output['Close/Last']).mean()
print('Mean absolute error is ')
print(mean_absolute_error)
abs(test_output['pred_Close/Last'] - test_output['Close/Last']).mean()/test_output['Close/Last'].mean()


,pred_Close/Last,Close/Last
Date,,
2025-03-25,"5,749.68","5,776.65"
2025-03-26,"5,776.81","5,712.20"
2025-03-27,"5,712.62","5,693.31"
2025-03-28,"5,716.67","5,580.94"
2025-03-31,"5,545.60","5,611.85"


Mean absolute error is 
55.036316456011974


0.009581567867516282

In [12]:
from statsmodels.tsa.api import VAR
model = VAR(X)
model_fit = model.fit()
var_forecast = model_fit.forecast(X.values, steps=4)
latest_data = pd.DataFrame(var_forecast, columns=X_train.columns)
latest_data = latest_data.fillna(0)
predictions = []
predicted_dates = [subset_data.index[-1] + BDay(i) for i in range(1, 5)]

for i in range(4):
    current_features = latest_data.iloc[[i]].fillna(0)  # Fill any remaining NaNs

    # Get predictions from models
    decision_pred = treemodel.predict(current_features)[0]
    lr_pred = closelast_model.predict(current_features)[0]
    
    # Apply a weighted average to stabilize predictions
    next_day_prediction = decision_pred + lr_pred
    predictions.append(next_day_prediction)

    # Shift the feature set while keeping some real values
    if i < 3:  # Only update if we have more predictions to make
        latest_data.iloc[i+1, -1] = next_day_prediction  # Set the latest prediction

# Print predictions
print("Predicted Closing Prices for the Next 4 Business Days:")
for i, pred in enumerate(predictions):
    print(f"{predicted_dates[i].date()}: {pred}")

Predicted Closing Prices for the Next 4 Business Days:
2025-04-01: 5594.063129116036
2025-04-02: 5595.289927179544
2025-04-03: 5632.152939450885
2025-04-04: 5620.805375033192


In [13]:
correct_predictions = [5776.65,5712.2,5693.31,5580.94] #for march 25,26,27,28 
error_sum = 0
for i in range(0,len(correct_predictions)):
    error = abs(correct_predictions[i] - predictions[i])
    print(error)
    error_sum += error
print("MAE Error is: ",error_sum/4)




182.58687088396346
116.91007282045575
61.15706054911516
39.86537503319232
MAE Error is:  100.12984482168167
